In [13]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [14]:
# Import the train and test dataset

train_df = pd.read_csv("train.csv", parse_dates=["first_active_month"]) #makes first_active_month column as datetime column 
test_df = pd.read_csv("test.csv", parse_dates=["first_active_month"])

print("Number of rows and columns in train set : ",train_df.shape)
print("Number of rows and columns in test set : ",test_df.shape)

Number of rows and columns in train set :  (201917, 6)
Number of rows and columns in test set :  (123623, 6)


In [15]:
# Created a elapsed time column for each id in train set:

import datetime
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['year'] = df['first_active_month'].dt.year
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df
train_df = read_data('train.csv')
test_df = read_data('test.csv')

target = train_df['target']
del train_df['target']

In [16]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,year,month,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,2017,6,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,2017,1,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,2016,8,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,2017,9,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,2017,11,92


In [17]:
# Now converting feature columns into categorical columns

train_df['feature_1'] = train_df['feature_1'].astype('category')
train_df['feature_2'] = train_df['feature_2'].astype('category')
train_df['feature_3'] = train_df['feature_3'].astype('category')

test_df['feature_1'] = test_df['feature_1'].astype('category')
test_df['feature_2'] = test_df['feature_2'].astype('category')
test_df['feature_3'] = test_df['feature_3'].astype('category')

In [18]:
print("Data types of Train set: ")
print(train_df.dtypes)

print( " ")
print("Data types of Test set: ")
print(test_df.dtypes)

Data types of Train set: 
first_active_month    datetime64[ns]
card_id                       object
feature_1                   category
feature_2                   category
feature_3                   category
year                           int64
month                          int64
elapsed_time                   int64
dtype: object
 
Data types of Test set: 
first_active_month    datetime64[ns]
card_id                       object
feature_1                   category
feature_2                   category
feature_3                   category
target                       float64
year                         float64
month                        float64
elapsed_time                 float64
dtype: object


In [19]:
# checking missing data for test dataset

total = test_df.isnull().sum().sort_values(ascending = False)
percent = (test_df.isnull().sum()/test_df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
elapsed_time,1,0.000809
month,1,0.000809
year,1,0.000809
first_active_month,1,0.000809
target,0,0.000000
feature_3,0,0.000000
feature_2,0,0.000000
feature_1,0,0.000000
card_id,0,0.000000


In [20]:
# Filling the missing value in test_df

test_df.loc[test_df['first_active_month'].isna(), 'first_active_month'] = test_df.loc[(test_df['feature_1'] == 5) & (test_df['feature_2'] == 2) & (test_df['feature_3'] == 1), 'first_active_month'].min()

__Checking for the missing values from the test_df and filling the missing values of ‘first_active_month’ column with the minimum month (in Timestamp) which satisfy the given condition on all the feature_1, feature_2 and feature_3 column values. These values for condition is chosen based on the data descriptions (manually found the row).__

In [21]:
# Now import the historical data

hist_df = pd.read_csv("historical_transactions.csv")
hist_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [22]:
# Calculating the missing values in historical dataset

total = hist_df.isnull().sum().sort_values(ascending = False)
percent = (hist_df.isnull().sum()/hist_df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
category_2,2652864,9.112500
category_3,178159,0.611970
merchant_id,138481,0.475678
subsector_id,0,0.000000
state_id,0,0.000000
purchase_date,0,0.000000
purchase_amount,0,0.000000
month_lag,0,0.000000
merchant_category_id,0,0.000000
installments,0,0.000000


__In this pre-processing step, the raw data is prepared in a polished way to be fed inside the network. So here, the ‘authorized_flag’ is converted to binary form where ‘Y’ corresponds to 1 and ‘N’ corresponds to 0. Until it is changed to binary format.__

In [23]:
# convert the authorized_flag to a binary value

hist_df['authorized_flag'] = hist_df['authorized_flag'].map({'Y':1, 'N':0})

Here we are calculating the aggerate functions of different columns in historical dataset. 

In [24]:
# Calculating the aggregate of variables

def aggregate_historical_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'purchase_date': [np.ptp],
        'month_lag': ['min', 'max']
        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['hist_' + '_'.join(col).strip() 
                           for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='hist_transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

history = aggregate_historical_transactions(hist_df)

In [25]:
# Merge history data and train data 

train_df = pd.merge(train_df, history, on='card_id', how='left')
test_df = pd.merge(test_df, history, on='card_id', how='left')
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,year,month,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,...,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_date_ptp,hist_month_lag_min,hist_month_lag_max
0,2017-06-01,C_ID_92a2005557,5,2,1,2017,6,245,260,247,...,-0.739395,0.212139,4,0.0,1,0,0.123314,20977987.0,-8,0
1,2017-01-01,C_ID_3d0044924f,4,1,0,2017,1,396,350,339,...,-0.742400,0.384967,543,1.0,10,-1,1.510777,33717687.0,-12,0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,2016,8,549,43,41,...,-0.730138,0.087380,0,0.0,0,0,0.000000,35635623.0,-13,0
3,2017-09-01,C_ID_186d6a6901,4,3,0,2017,9,153,77,77,...,-0.740897,0.261624,84,1.0,3,-1,0.588974,13375339.0,-5,0
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,2017,11,92,133,128,...,-0.746156,1.352094,182,1.0,12,1,1.896862,9405641.0,-3,0


### to do: filling the null values in new and old merchant and historical dataset

In [26]:
# Import old Merchant data

merchant_df = pd.read_csv("merchants.csv")
print("shape of merchant : ",merchant_df.shape)

shape of merchant :  (334696, 22)


In [30]:
merchant_df.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [29]:
# checking missing data of old merchant data

total = merchant_df.isnull().sum().sort_values(ascending = False)
percent = (merchant_df.isnull().sum()/merchant_df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
category_2,11887,3.551581
avg_sales_lag3,13,0.003884
avg_sales_lag12,13,0.003884
avg_sales_lag6,13,0.003884
merchant_group_id,0,0.000000
merchant_category_id,0,0.000000
subsector_id,0,0.000000
numerical_1,0,0.000000
numerical_2,0,0.000000
category_1,0,0.000000


In [31]:
# Import new merchant data file

new_merchant_df = pd.read_csv("new_merchant_transactions.csv")
print("shape of new_merchant_transactions : ",new_merchant_df.shape)

shape of new_merchant_transactions :  (1048575, 14)


In [32]:
# Calculating the missing values in new merchant dataset

total = new_merchant_df.isnull().sum().sort_values(ascending = False)
percent = (new_merchant_df.isnull().sum()/new_merchant_df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
category_2,52936,5.048375
category_3,26054,2.484705
merchant_id,12044,1.148606
subsector_id,0,0.000000
state_id,0,0.000000
purchase_date,0,0.000000
purchase_amount,0,0.000000
month_lag,0,0.000000
merchant_category_id,0,0.000000
installments,0,0.000000


In [33]:
# convert the authorized_flag to a binary value

new_merchant_df['authorized_flag'] = new_merchant_df['authorized_flag'].map({'Y':1, 'N':0})

In [34]:
def aggregate_new_transactions(new_trans):    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'month_lag': ['min', 'max']
        }
    agg_new_trans = new_trans.groupby(['card_id']).agg(agg_func)
    agg_new_trans.columns = ['new_' + '_'.join(col).strip() 
                           for col in agg_new_trans.columns.values]
    agg_new_trans.reset_index(inplace=True)
    
    df = (new_trans.groupby('card_id')
          .size()
          .reset_index(name='new_transactions_count'))
    
    agg_new_trans = pd.merge(df, agg_new_trans, on='card_id', how='left')
    
    return agg_new_trans

new_merchant_df = aggregate_new_transactions(new_merchant_df)

In [35]:
# Printing the New Merchant data 

new_merchant_df.head()

,card_id,new_transactions_count,new_authorized_flag_sum,new_authorized_flag_mean,new_merchant_id_nunique,new_city_id_nunique,new_purchase_amount_sum,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_median,new_installments_max,new_installments_min,new_installments_std,new_month_lag_min,new_month_lag_max
0,C_ID_0001238066,26,26,1,25,8,-14.850055,-0.649235,-0.078318,-0.740897,0.173436,42,1.0,10,-1,2.079941,1,2
1,C_ID_0001793786,31,31,1,31,7,-0.229620,-0.372748,3.129932,-0.737892,0.947223,0,0.0,0,0,0.000000,1,2
2,C_ID_000183fdda,11,11,1,11,2,-6.590778,-0.665765,-0.107680,-0.732332,0.182877,16,1.0,4,-1,1.293340,1,2
3,C_ID_00032df08f,8,8,1,7,1,-2.791535,-0.478579,0.449203,-0.640520,0.366423,8,1.0,1,1,0.000000,2,2
4,C_ID_00057b99fe,1,1,1,1,1,-0.701828,-0.701828,-0.701828,-0.701828,NaN,0,0.0,0,0,NaN,1,1


In [36]:
# Merge train data and new merchant data:

train_df = pd.merge(train_df, new_merchant_df, on='card_id', how='left')
test_df = pd.merge(test_df, new_merchant_df, on='card_id', how='left')
train_df.shape

(201917, 43)

In [37]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,year,month,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,...,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_median,new_installments_max,new_installments_min,new_installments_std,new_month_lag_min,new_month_lag_max
0,2017-06-01,C_ID_92a2005557,5,2,1,2017,6,245,260,247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,C_ID_3d0044924f,4,1,0,2017,1,396,350,339,...,-0.701858,-0.739410,0.014326,6.0,1.0,1.0,1.0,0.000000,1.0,2.0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,2016,8,549,43,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-09-01,C_ID_186d6a6901,4,3,0,2017,9,153,77,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,2017,11,92,133,128,...,0.450886,-0.739395,0.223821,35.0,1.0,2.0,-1.0,0.376913,1.0,2.0


In [38]:
# Printing the categories of the variables

use_cols = [col for col in train_df.columns if col not in ['card_id', 'first_active_month']]

train_df = train_df[use_cols]
test_df = test_df[use_cols]

features = list(train_df[use_cols].columns)
categorical_feats = [col for col in features if 'feature_' in col]

for col in categorical_feats:
    print(col, 'have', train_df[col].value_counts().shape[0], 'categories.')

feature_1 have 5 categories.
feature_2 have 3 categories.
feature_3 have 2 categories.


__It’s trying to find out the number of categories every feature is having.__

In [39]:
# Changing the categorical column into numerical form

from sklearn.preprocessing import LabelEncoder
for col in categorical_feats:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

feature_1
feature_2
feature_3


__Since all the features columns have different class, the label encoder is generated for each classes and changed it to the numbers.__

In [40]:
# Merge train data and test data

df_all = pd.concat([train_df, test_df])
df_all = pd.get_dummies(df_all, columns=categorical_feats)

len_train_df = train_df.shape[0]

train = df_all[:len_train_df]
test = df_all[len_train_df:]
train.shape

(201917, 48)

In [41]:
# Printing the head of the final train dataset

train.head()

,year,month,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_merchant_id_nunique,hist_city_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_median,...,feature_1_0,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_2_0,feature_2_1,feature_2_2,feature_3_0,feature_3_1
0,2017.0,6.0,245.0,260,247,0.950000,94,7,-165.968739,-0.698042,...,0,0,0,0,1,0,1,0,0,1
1,2017.0,1.0,396.0,350,339,0.968571,142,9,-210.006336,-0.708590,...,0,0,0,1,0,1,0,0,1,0
2,2016.0,8.0,549.0,43,41,0.953488,13,5,-29.167391,-0.698868,...,0,1,0,0,0,0,1,0,1,0
3,2017.0,9.0,153.0,77,77,1.000000,50,7,-49.491364,-0.707989,...,0,0,0,1,0,0,0,1,1,0
4,2017.0,11.0,92.0,133,128,0.962406,66,6,-48.687656,-0.689807,...,1,0,0,0,0,0,0,1,1,0


__Since all the preprocessing are done. The train data is now merged to test data. Now we have 48 variables all together after preprocessing and merging the historical, merchant and train datasets.__

In [42]:
corr = train.corr()
corr

,year,month,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_merchant_id_nunique,hist_city_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_median,...,feature_1_0,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_2_0,feature_2_1,feature_2_2,feature_3_0,feature_3_1
year,1.000000,-0.119644,-0.938988,-0.092920,-0.087409,0.007952,-0.059279,-0.014634,0.001545,0.008370,...,0.088519,0.115044,-0.130126,0.079872,-0.083762,0.100575,0.029293,-0.164255,0.194084,-0.194084
month,-0.119644,1.000000,-0.229130,-0.101082,-0.096008,0.030220,-0.096375,-0.057505,-0.001982,0.003735,...,0.037731,-0.000749,-0.020847,-0.000937,0.004273,0.041696,-0.043482,0.000752,0.016784,-0.016784
elapsed_time,-0.938988,-0.229130,1.000000,0.126030,0.118870,-0.018283,0.091434,0.034233,-0.000825,-0.009503,...,-0.099886,-0.112443,0.134794,-0.078010,0.080586,-0.112936,-0.013779,0.160785,-0.196046,0.196046
hist_transactions_count,-0.092920,-0.101082,0.126030,1.000000,0.994572,0.149331,0.859495,0.561965,-0.004046,-0.109555,...,0.123048,-0.067749,-0.057878,0.035862,0.045724,-0.052114,0.109418,-0.069104,0.019213,-0.019213
hist_authorized_flag_sum,-0.087409,-0.096008,0.118870,0.994572,1.000000,0.212267,0.865707,0.566279,-0.004092,-0.109118,...,0.138258,-0.069417,-0.063201,0.047265,0.036516,-0.048001,0.103933,-0.067549,0.031827,-0.031827
hist_authorized_flag_mean,0.007952,0.030220,-0.018283,0.149331,0.212267,1.000000,0.218843,0.149884,-0.000701,-0.081984,...,0.147387,-0.025587,-0.086408,0.133141,-0.053734,0.015911,-0.014002,-0.002919,0.127352,-0.127352
hist_merchant_id_nunique,-0.059279,-0.096375,0.091434,0.859495,0.865707,0.218843,1.000000,0.738163,-0.003910,-0.116529,...,0.191111,-0.080486,-0.090430,0.075142,0.029618,-0.021792,0.071723,-0.061038,0.063844,-0.063844
hist_city_id_nunique,-0.014634,-0.057505,0.034233,0.561965,0.566279,0.149884,0.738163,1.000000,-0.003418,-0.049349,...,0.155055,-0.064598,-0.127162,0.067265,0.083139,0.054625,-0.005127,-0.063174,0.056220,-0.056220
hist_purchase_amount_sum,0.001545,-0.001982,-0.000825,-0.004046,-0.004092,-0.000701,-0.003910,-0.003418,1.000000,0.000260,...,-0.000971,-0.000972,0.002593,-0.000852,-0.000822,0.001912,-0.000466,-0.001857,-0.001853,0.001853
hist_purchase_amount_median,0.008370,0.003735,-0.009503,-0.109555,-0.109118,-0.081984,-0.116529,-0.049349,0.000260,1.000000,...,0.054982,-0.047478,-0.154362,0.041052,0.175279,0.089131,-0.069469,-0.027452,0.008107,-0.008107
